In [1]:
# Ввод необходимых библиотек
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import enum 
import scipy.optimize as optimize
import datetime
import statsmodels.api as sm
import pandas as pd
#!pip install yahoo_fin
#from yahoo_fin import options
import time
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Вводим мнимую единицу i
np.set_printoptions(precision = 5)
i   = np.complex(0.0,1.0)

# Вводим класс отражающий тип опциона

class OptionType(enum.Enum):
    CALL = 1.0
    PUT = -1.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def CallPutOptionPriceCOSMthd(par, CP, S0, r, tau, K, N, L):
    # par  - параметры модели
    # CP   - тип опциона
    # S0   - начальная цена базового актива
    # r    - процентная ставка (постоянная)
    # tau  - время до экспирации
    # K    - лист страйков
    # N    - число элементов в разложении
    # L    - определяет величину отсечения пространства (L=8 или L=10)
        
    
    # меняем размер K - теперь K вектор
    if K is not np.array:
        K = np.array(K).reshape([len(K), 1])
    
    i = np.complex(0.0, 1.0) 
    x0 = np.log(S0 / K)   
    
    # усечённое пространство

    a = 0.0 - L * np.sqrt(tau)
    b = 0.0 + L * np.sqrt(tau)
    
    # Суммирование по k = 0 до k = N-1

    k = np.linspace(0, N-1, N).reshape([N, 1])  
    u = k * np.pi / (b - a);  

    # определяем коэффициенты и характеристическую функцию
    cf = ChFHestonModel(r,tau, par)
    H_k = CallPutCoefficients(CP,a,b,k)   
    mat = np.exp(i * np.outer((x0 - a) , u))
    temp = cf(u) * H_k 
    temp[0] = 0.5 * temp[0] 
    # суммируем полученные коэффициенты   
    value = np.exp(-r * tau) * K * np.real(mat.dot(temp))     
    return value


In [ ]:
# вычисление коэффициентов для соответствующего типа опционов
def CallPutCoefficients(CP,a,b,k):
    if CP==OptionType.CALL:                  
        c = 0.0
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        if a < b and b < 0.0:
            H_k = np.zeros([len(k),1])
        else:
            H_k = 2.0 / (b - a) * (Chi_k - Psi_k)  
    elif CP==OptionType.PUT:
        c = a
        d = 0.0
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k)               
    
    return H_k    


In [ ]:
# вспомогательная функция для вычисления коэффициентов в сумме
def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value
    

In [ ]:
# формула Блэка - Шоулса для стоимости опциона

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    
    K = np.array(K).reshape([len(K),1])
    sigma = np.array(sigma).reshape([len(sigma),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) 
    * tau) / (sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if CP == OptionType.CALL:
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif CP == OptionType.PUT:
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value


In [ ]:
# вычисление подразумеваемой волатильности по цене опциона, страйку, времени до экспирации
# в качестве метода поиска корня используется метод Ньютона Рафсона
def ImpliedVolatility(CP,marketPrice,K,T,S_0,r):
    func = lambda sigma: np.power(BS_Call_Option_Price(CP,S_0,K,sigma,T,r) - marketPrice,1.0)
    impliedVol = optimize.newton(func, 0.7, tol=1e-5)
    return impliedVol


In [ ]:
# вычисление характеристической функции в модели Хестона
def ChFHestonModel(r,tau,par):

    kappa = par[0]
    gamma = par[1]
    vbar = par[2]
    v0 = par[3]
    rho = par[4]

    i = np.complex(0.0,1.0)
    D1 = lambda u: np.sqrt(np.power(kappa-gamma*rho*i*u,2)+(u*u+i*u)*gamma*gamma)
    g  = lambda u: (kappa-gamma*rho*i*u-D1(u))/(kappa-gamma*rho*i*u+D1(u))
    C  = lambda u: (1.0-np.exp(-D1(u)*tau))/(gamma*gamma*(1.0-g(u)*D1(u)*tau)) \
        *(kappa-gamma*rho*i*u-D1(u))

    # тут нет множителя -r*tau, так как дисконтирование происходит в функции для COS метода

    A  = lambda u: r * i*u *tau + kappa*vbar*tau/gamma/gamma *(kappa-gamma*rho*i*u-D1(u))\
        - 2*kappa*vbar/gamma/gamma*np.log((1-g(u)*np.exp(-D1(u)*tau))/(1-g(u)))

    # характеристическая функция в модели Хестона    

    cf = lambda u: np.exp(A(u) + C(u)*v0)
    return cf 


In [ ]:
# генератор путей методом Эйлера с поглощением в 0 вариации в одномерной модели Хестона
def GeneratePathsHestonEuler(NoOfPaths, NoOfSteps, T, r, S_0, kappa, gamma,  vbar, v0, rho):  
    # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)  
    Z1 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    Z2 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    W1 = np.zeros([NoOfPaths, NoOfSteps+1])
    W2 = np.zeros([NoOfPaths, NoOfSteps+1])
    V = np.zeros([NoOfPaths, NoOfSteps+1])
   
    X = np.zeros([NoOfPaths, NoOfSteps+1])
   
    V[:,0]=v0
    X[:,0]=np.log(S_0)
    time = np.zeros([1, NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):

        # убеждаемся, что элементы выборки из нормального распределения со средним 0 и дисперсией 1

        if NoOfPaths > 1:
            Z1[:,i] = (Z1[:,i] - np.mean(Z1[:,i])) / np.std(Z1[:,i])
            Z2[:,i] = (Z2[:,i] - np.mean(Z2[:,i])) / np.std(Z2[:,i])
        Z2[:,i] = rho * Z1[:,i] + np.sqrt(1.0-rho**2)*Z2[:,i]
        
        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        
       

        V[:,i+1] = V[:,i] + kappa*(vbar - V[:,i]) * dt + gamma* np.sqrt(V[:,i]) * (W1[:,i+1]-W1[:,i])
       
        # если вариация стала отрицательной, то она поглощается в 0
        V[:,i+1] = np.maximum(V[:,i+1],0.0)
        X[:,i+1] = X[:,i] + (r - 0.5*V[:,i])*dt + np.sqrt(V[:,i])*(W2[:,i+1]-W2[:,i])
        
        time[0][i+1] = time[0][i] +dt
        
    # Compute exponent

    S = np.exp(X)
    paths = {"time":time,"S":S}
    return paths


  

In [2]:
def CIR_Sample(NoOfPaths, kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    sample = c* np.random.noncentral_chisquare(delta,kappaBar,NoOfPaths)
    return  sample
# моделирование путей в модели Хестона с помощью почти точной схемы симуляции
def GeneratePathsHestonAES(NoOfPaths, NoOfSteps, T, r, S_0, kappa, gamma, vbar, v0, rho): 
    # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)   
    Z1 = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    W1 = np.zeros([NoOfPaths, NoOfSteps+1])
    V = np.zeros([NoOfPaths, NoOfSteps+1])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    V[:,0]=v0
    X[:,0]=np.log(S_0)
    
    time = np.zeros([NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):

        # making sure that samples from a normal have mean 0 and variance 1

        if NoOfPaths > 1:
            Z1[:,i] = (Z1[:,i] - np.mean(Z1[:,i])) / np.std(Z1[:,i])
        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        
        # Exact samples for the variance process

        V[:,i+1] = CIR_Sample(NoOfPaths,kappa,gamma,vbar,0,dt,V[:,i])
        k0 = (r -rho/gamma*kappa*vbar)*dt
        k1 = (rho*kappa/gamma -0.5)*dt - rho/gamma
        k2 = rho / gamma
        X[:,i+1] = X[:,i] + k0 + k1*V[:,i] + k2 *V[:,i+1] + np.sqrt((1.0-rho**2)*V[:,i])*(W1[:,i+1]-W1[:,i])
        time[i+1] = time[i] +dt
        
    # Compute exponent

    S = np.exp(X)
    paths = {"time":time.reshape([1, len(time)]),"S":S}
    return paths

In [3]:
# моделирование двухмерной коррелированной модели Хестона на основе моделирования методом Эйлера
# возвращает 2 путя
def Heston2path(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12):
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)
  # мгновенная кореляция между активами
  Z1 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z2 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z3 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z4 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  W1 = np.zeros([1, NoOfSteps + 1])
  W2 = np.zeros([1, NoOfSteps + 1])
  W3 = np.zeros([1, NoOfSteps + 1])
  W4 = np.zeros([1, NoOfSteps + 1])
  V1 = np.zeros([1, NoOfSteps + 1])
  V2 = np.zeros([1, NoOfSteps + 1])
  X1 = np.zeros([1, NoOfSteps + 1])
  X2 = np.zeros([1, NoOfSteps + 1])
  V1[:,0] = par1[3]
  V2[:,0] = par2[3]
  X1[:,0] = np.log(S_01)
  X2[:,0] = np.log(S_02)
  time = np.zeros([1, NoOfSteps+1])    
  dt = T / float(NoOfSteps)
  for i in range(0,NoOfSteps):
       
        Z2[:,i] = par1[4] * Z1[:,i] + np.sqrt(1.0-par1[4]**2) * Z2[:,i]
        Z4[:,i] = par2[4] * rho12 * Z1[:,i] + par2[4] * np.sqrt(1.0-rho12**2) * Z3[:,i] + np.sqrt(1.0-par2[4] ** 2) * Z4[:,i]
        Z3[:,i] = rho12 * Z1[:,i] + np.sqrt(1.0-rho12**2) * Z3[:,i]

        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        W3[:,i+1] = W3[:,i] + np.power(dt, 0.5)*Z3[:,i]
        W4[:,i+1] = W4[:,i] + np.power(dt, 0.5)*Z4[:,i]
        
        
        V1[:,i+1] = V1[:,i] + par1[0] * (par1[2] - V1[:,i]) * dt + par1[1] * np.sqrt(V1[:,i]) * (W2[:,i+1]-W2[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1],0.0)
        X1[:,i+1] = X1[:,i] + (mu[0] - 0.5 * V1[:,i]) * dt + np.sqrt(V1[:,i]) * (W1[:,i+1]-W1[:,i])
        V2[:,i+1] = V2[:,i] + par2[0] * (par2[2] - V2[:,i]) * dt + par2[1] * np.sqrt(V2[:,i]) * (W4[:,i+1]-W4[:,i])
        V2[:,i+1] = np.maximum(V2[:,i+1],0.0)
        X2[:,i+1] = X2[:,i] + (mu[1] - 0.5 * V2[:,i]) * dt + np.sqrt(V2[:,i]) * (W3[:,i+1]-W3[:,i])
        
        time[0][i+1] = time[0][i] + dt
  return np.exp(X1), np.exp(X2), time    



In [4]:
# функция для подсчёта выборочной корреляции между двумя путями
def EmpCorr(path1, path2):
  return np.corrcoef([np.log(path1[j + 1]/path1[j]) for j in range(len(path1) - 1)], [np.log(path2[j + 1]/path2[j]) for j in range(len(path2) - 1)])[0][1]


In [5]:
# функция для подсчёта ожидаемой выборочной корреляции в случае модели Хестона
def E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12):
  s = 0
  for i in range(N):
    a = Heston2path(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12)
    s += EmpCorr(np.array(a[0][0]), np.array(a[1][0]))
  return s / N

In [6]:
def corrHestoncalibration(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp, bounds):
  def err_fun(x):
    return (E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, x) - rho_emp) ** 2
  res = optimize.differential_evolution(err_fun, bounds, atol = 1e-2)
  return res.x[0]

In [7]:
# калибровка(на основе метода бисекций) для вычисления мгновенной корреляции между двумя активами при данной исторической корреляции и откалиброванными параметрами модели Хестона
def Bis_opt(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp, eps):
    # N - количество слагаемых для вычисления матожидания
    # par1, par2 - откалиброванные параметры модели Хестона для 1 и 2 актива соответственно
    # NoOfSteps - количество шагов в моделировании путей
    # S_01, S_02 - начальные цены активов
    # rho_emp - историческая корреляция
    # eps - допустимая погрешность
    rho_l = -1
    rho_u = 1
    E_l = E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_l) - rho_emp
    rho_emp_new = 0.5 * (rho_l + rho_u)
    E_delta = E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp_new) - rho_emp
    if (rho_emp > 0.99 and rho_emp <= 1) or (rho_emp < -0.99 and rho_emp >= -1.0):
      eps = 1e-2
    
    while abs(E_delta) > eps:
      if E_l * E_delta < 0:
        rho_u = rho_emp_new
        rho_emp_new = 0.5 * (rho_l + rho_u)
        E_delta = E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp_new) - rho_emp
        #print(rho_emp_new)
      else:
        rho_l = rho_emp_new
        E_l = E_delta
        rho_emp_new = 0.5 * (rho_l + rho_u)
        E_delta = E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp_new) - rho_emp
       # print(rho_emp_new)
    return rho_emp_new


In [8]:
# калибровка матрицы корреляций
# на входе матрица исторических корреляций между активами 
# на выходе откалиброванная матрица мгновенных корреляций

def corr_calibration(par_model, par_S_0, N, NoOfSteps, T, mu, rho_emp_matr):
  # par_model - матрица, строка i которой состоит из откалиброванных параметров модели Хестона для актива i
  # par_S_0 - вектор начальных стоимостей активов
  # rho_emp_matr - матрица исторических корреляций

  corr_Xi_Xj = np.zeros([len(par_S_0), len(par_S_0)])
  bounds = [(-1.0, 1.0)]
  eps = 10e-3
  # вычисляем матрицу мгновенных корреляций между активами
  for i in range(len(par_S_0)):
    corr_Xi_Xj[i][i] = 1
    for j in range(i + 1, len(par_S_0)):
      #corr_Xi_Xj[i][j] = corrHestoncalibration(N, par_model[i], par_model[j], NoOfSteps, T, np.array([mu[i], mu[j]]), par_S_0[i], par_S_0[j], rho_emp_matr[i][j], bounds)
      corr_Xi_Xj[i][j] = Bis_opt(N, par_model[i], par_model[j], NoOfSteps, T, np.array([mu[i], mu[j]]), par_S_0[i], par_S_0[j], rho_emp_matr[i][j], eps)
      corr_Xi_Xj[j][i] = corr_Xi_Xj[i][j]
  return corr_Xi_Xj
  

In [9]:
def check_pos_def(vec):
  for i in vec:
    if i <= 0:
      return False
  return True

In [10]:
def chol(M):
  L = np.zeros([len(M[0]), len(M[0])])
  for j in range(len(M[0])):
    for i in range(j, len(M[0])):
      if i == 0 and j == 0:
        L[i, j] = np.sqrt(M[i, j])
      if i == j and i != 0 and j != 0:
        if M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])) <= 0:
          L[i, j] = 10e-5
        else:
          L[i, j] = np.sqrt(M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])))
          
      else:
        L[i, j] = (M[i, j] - np.sum(np.array([L[j, k] * L[i, k ] for k in range(j)])))/L[j, j]
  return L

In [11]:
# вычисление разложения Холецкого для общей корреляционной структуры
# на входе параметры моделей в виде двумерного листа и матрица корреляций между броуновскими движениями активов
def chol_matr(par_model, corr_matr):
  C = np.zeros([len(par_model) * 2, len(par_model) * 2])
  # конструируем общую корреялционную матрицу между броуновскими движениями в системе
  for i in range(len(par_model)):
    C[i, i] = 1
    C[i, i + len(par_model)] = par_model[i][4]
    C[i + len(par_model), i ] = C[i, i + len(par_model)] 
  for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
    C[i, i] = 1
    for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
      
      C[i, j] = corr_matr[k1][k2]
      C[j, i] = C[i, j]
 
  w, v = np.linalg.eig(C)
  
  # проверяем положительную определённость, если не выполняется, то используем метод регуляризации Джэкеля
  if check_pos_def(w) == True:
   
    return np.linalg.cholesky(C)
  else:
    Lambda = np.zeros([len(w), len(w)])
    sp = np.zeros([len(w)])
    for i in range(len(w)):
      if w[i] >= 0:
        Lambda[i][i] = w[i]
        sp[i] = w[i] 
    T = np.zeros([len(w), len(w)])
    for i in range(len(w)):
      T[i][i] = 1/(np.power(v[i], 2).dot(sp))
    
    B = np.dot(np.dot(np.sqrt(T), v), np.sqrt(Lambda))
    Cnew = np.dot(B, np.transpose(B))
    w, v = np.linalg.eig(Cnew)
    for i in range(len(par_model) *2):
        Cnew[i, i] = 1
    return chol(Cnew)

      

In [12]:


def FirstApproach(m, s2):
    b2 = 2.0 * m * m / s2 - 1.0 + np.sqrt(2.0 * m * m / s2)*np.sqrt((2.0 * m * m / s2) - 1.0)
    b  = np.sqrt(b2)
    a  = m  / (1.0 + b2)
    return a,b

def SecondApproach(m, s2):
    c = ((s2 / (m*m)) - 1.0) / ((s2 / (m*m)) + 1.0)
    d = (1.0 - c) / m
    return c, d
# вычисление параметров для QE схемы
def CIRCDF(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    cdf =lambda x: st.ncx2.cdf(x/c,delta,kappaBar)
    return cdf

def CIRMean(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(- kappa * (t-s)))
    kappaBar = 4.0 * kappa * v_s * np.exp(-kappa * (t-s)) / (gamma * gamma * (1.0 - np.exp(-kappa*(t-s))))
    return c * (delta + kappaBar)

def CIRVar(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    VarV = c*c*(2.0*delta+4.0*kappaBar)
    return VarV


In [13]:
# QE схема для моделирования  v(t)|v(s)
def QE_scheme(kappa, gamma, vbar, s, t, v_s, NoOfSamples):
    m  = CIRMean(kappa, gamma, vbar, s, t, v_s)
    s2 = CIRVar(kappa, gamma, vbar, s, t, v_s)
    if m < 0 or s2 < 0:
      print('параметры', kappa, gamma, vbar, s, t, v_s)
    aStar = 1.5 # параметр разделения
    if (s2/ (m * m) < aStar):

        # a и b - первый подход
        b1 = 2.0 * m * m / s2 - 1.0 + np.sqrt(2.0 * m * m / s2)*np.sqrt((2.0 * m * m / s2) - 1.0)
        b  = np.sqrt(b1)
        a  = m  / (1.0 + b1)
        Z = np.random.normal(0.0, 1.0, [NoOfSamples, 1])
        
        if NoOfSamples > 1:
          Z = (Z - np.mean(Z)) / np.std(Z)
        
        A = a * np.power(b + Z, 2.0)

    else:

        # c и d - второй подход
        c = ((s2 / (m*m)) - 1.0) / ((s2 / (m*m)) + 1.0)
        d = (1.0 - c) / m
        U = np.random.uniform(0.0, 1.0, [NoOfSamples, 1])
        A = 1.0 / d * np.log((1.0-c)/(1.0-U))
        A[U < c] = 0.0
   
    return A  
    

In [14]:
# Моделирование многомерного пути в модели Хестона методом MQE

def multi_asset_path(par_model, corr_matrix, parS0, T, r, NoOfSteps):
   # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)   
   # par_model - двумерный лист параметров 
   # corr_matrix - откалиброванная мтариц корреляции между броуновскими движениями активов
   # parS0 - лист начальных стоимостей активов
   # T - дата экспирации
   # r - процентная ставка
   # NoOfSteps - количество шагов
   
    V = np.zeros([len(parS0),  NoOfSteps+1])
    X = np.zeros([len(parS0),  NoOfSteps+1])
    Vint = np.zeros([len(parS0),  NoOfSteps+1])
    f = np.zeros([len(parS0), 1])
    s = np.zeros([len(parS0), 1])
    L = chol_matr(par_model, corr_matrix)
    rho_m = np.zeros([len(parS0), len(parS0)])

    for i in range(len(parS0)):
      rho_m[i, i] = par_model[i][4]

    Ds = np.zeros([len(parS0), len(parS0)])
    for i in range(len(parS0)):
      V[i, 0] = par_model[i][3]
      X[i, 0] = np.log(parS0[i])
    L_star = np.zeros([len(parS0), len(parS0)])
    for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
      for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
          L_star[k1, k2] = L[i, j]
      
    time = np.zeros([NoOfSteps+1])
    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):
        time[i+1] = time[i] + dt
        Z = np.random.normal(0.0, 1.0, [len(parS0), 1])
        for j in range(len(parS0)):
            
            V[j,i+1] = QE_scheme(par_model[j][0], par_model[j][1], par_model[j][2], time[i], time[i+1], V[j, i], 1)[0][0]
            Vint[j, i] = dt * (V[j,i+1] + V[j,i])/2
            f[j] = (V[j,i+1] - V[j,i] - par_model[j][0] * par_model[j][2] * dt + par_model[j][0] * Vint[j, i]) / par_model[j][1]
            Ds[j, j] = np.sqrt(Vint[j, i])
        
        
        
        X[:, i+1] = X[:, i] + (r  - Vint[:, i]/2) * dt + np.dot(rho_m , f).reshape([1, len(parS0)])[0] + np.dot(Ds, np.dot(L_star, Z)).reshape([1, len(parS0)])[0]
    return np.exp(X), time


In [15]:
def findind(arr, elem, par = 4):
  for i, j in enumerate(arr):
    if round(j, par) == elem:
      return i
  return False 

In [16]:
def autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N, dates, coupon, barier, autocall, nom, ins):
  price = []
  count = 0
  for i in range(N):
    C = 0
    memory = 0
    paths = multi_asset_path(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps)
    for k in range(len(par_S_0)):
      paths[0][k, :] = paths[0][k, :] / par_S_0[k]
    for l, j in enumerate(dates):
        d = min(paths[0][:, findind(paths[1], j)])
        if d < barier and l == len(dates) - 1:
          C += np.exp(- r * j) * d * nom / barier 
          count += 1
        elif barier <= d <= autocall and l == len(dates) - 1:
          memory += 1
          C += np.exp(- r * j)*(memory * nom * coupon / 100 + (1 - ins) * nom)
        elif d > autocall and l == 0:
          C = np.exp(- r * j)*(2 * nom * coupon / 100 + nom)
          break
        elif d > autocall:
          memory += 1
          C += np.exp(- r * j)*(memory * nom * coupon / 100 + nom)
          break
        elif d < barier:
          memory += 1
        elif barier <= d <= autocall:
          memory += 1
          C += np.exp(- r * j) * memory * nom * coupon / 100
          memory = 0
    price.append(C)
  price = np.array(price)
  z = 1.96
  m = np.mean(price)
  s = np.std(price, ddof=1)
  return m  , round(count / N, 3), m - z * s / np.sqrt(N) , m + z * s / np.sqrt(N)

In [ ]:
# Вводится тикер
# Возвращает: 
# 1) Цена закрытия последнего торгового дня
# 2) Таблица цен опционов (строки соответствуют экспирации, а столбцы страйкам)
# 3) Экспирации приведённые к виду тау
# 4) Страйки
def option_data(ticker):
  ticker_dates = options.get_expiration_dates(ticker)
  ticker_dates = ticker_dates[-10:]
  calls = {}
  for i in ticker_dates:
    calls.update({'{0}'.format(i) : options.get_options_chain(ticker, i)['calls']})

  data = []
  for i in ticker_dates:
    data.append([i, calls[i]['Strike'], calls[i]['Last Price']])

  strikes = set.intersection(*[set(i[1]) for i in data])

  def findind(arr, elem, par = 4):
    for i, j in enumerate(arr):
      if round(j, par) == elem:
        return i
    return False 

  final_data = []

  for i in data:
    help_data = []
    for j in sorted(list(strikes)): 
      help_data.append(i[2][findind(i[1], j)])

    final_data.append(help_data)

  interval = '1d'
  period1 = int(time.mktime(datetime.datetime(2022, 4, 14, 23, 59).timetuple()))
  period2 = int(time.mktime(datetime.datetime.now().timetuple())) 
  query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
  df = pd.read_csv(query_string)
  return {'Ticker': ticker, 'S0':float(df[-1:]['Close']), 'Option data': np.array(final_data), 'Expirations':np.array([(datetime.datetime.strptime(i, "%B %d, %Y") - datetime.datetime.now()).days/365 for i in ticker_dates]), 'Strikes':np.array(sorted(list(strikes))).reshape([len(strikes),1])}

In [ ]:
def Calibration_Heston2(CP, S0, r, N, L, x0, bounds, optimizer, market, expiration):
    # CP   - тип опциона
    # S0   - начальная цена базового актива
    # r    - процентная ставка (постоянная)
    # N    - число элементов в разложении
    # L    - определяет величину отсечения пространства (L=8 или L=10)
    # bounds - границы для параметров
    # optimizer - тип оптимизации : в данной работе применяется дифференциальная эволюция
    # market - данные, по которым калибруют (даны в виде подразумеваемых волатильностей)
    # expiration - вектор времён до экспирации
    # strikes - вектор страйков
    # функция ошибок 
    def Err_fun(x):
        # внутри суммирование идёт по страйкам при фиксированной экспирации, а далее идёт суммирвоание по экспирациям
        return np.sum(np.array([np.sum((CallPutOptionPriceCOSMthd([0.5, x[0], x[1], x[2], x[3]], CP, S0, r, expiration[i], market[i][1], N, L).reshape([1,len(market[i][1])]) - np.array(market[i][2]).reshape([1,len(market[i][1])])) ** 2) for i in range(len(expiration))]))
    # оптимизация при заданных границах
    res = optimizer(fun=Err_fun, x0=x0, method='Nelder-Mead', bounds=bounds)
    return res.x, res.fun

In [ ]:
def option_data2(ticker):
  ticker_dates = options.get_expiration_dates(ticker)
  ticker_dates = ticker_dates[-10:]
  calls = {}
  for i in ticker_dates:
    calls.update({'{0}'.format(i) : options.get_options_chain(ticker, i)['calls']})

  data = []
  for i in ticker_dates:
    data.append([i, np.array(calls[i]['Strike']).reshape([len(calls[i]['Strike']),1]), calls[i]['Last Price']])

  interval = '1d'
  period1 = int(time.mktime(datetime.datetime(2022, 4, 14, 23, 59).timetuple()))
  period2 = int(time.mktime(datetime.datetime.now().timetuple())) 
  query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
  df = pd.read_csv(query_string)
  return {'Ticker': ticker, 'S0':float(df[-1:]['Close']), 'Option data': data, 'Expirations':np.array([(datetime.datetime.strptime(i, "%B %d, %Y") - datetime.datetime.now()).days/365 for i in ticker_dates])}

In [ ]:
import pickle
tickers = ['MA']
for i in tickers:
  data = option_data2(i)
  a_file = open(f"{i}10.05.pkl", "wb")
  pickle.dump(data, a_file)
  a_file.close()

#a_file = open("data.pkl", "rb")
#output = pickle.load(a_file)

In [ ]:
tickers = ['PANW', 'STLD','NVDA','AMD','QCOM', 'MA']
CP  = OptionType.CALL
r   = 0.0078
N   = 10000
L   = 12
bounds = [(0.01, 1.0), (0.01, 1.0), (0.01, 1.0), (-1.0, 1)]
x0 = [0.3, 0.3, 0.2, -0.7]
for i in tickers:
  a_file = open(f"{i}10.05.pkl", "rb")
  data = pickle.load(a_file)
  a_file.close()
  #optimize.differential_evolution
  par = Calibration_Heston2(CP, data['S0'], r, N, L, x0, bounds,optimize.minimize, data['Option data'], data['Expirations'])
  print(i, par[0], par[1])
  a_file = open(f"{i}10.05.pkl", "wb")
  data.update({'Parameters Heston': par[0]})
  pickle.dump(data, a_file)
  a_file.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


PANW [ 0.56598  0.75425  0.77404 -1.     ] 1184193.3614621125
STLD [ 0.01     0.09771  0.03024 -0.6284 ] 3779.9492503283527
NVDA [0.64781 0.02785 1.      0.     ] 3863361.159493167
AMD [0.15895 1.      1.      0.     ] 185678.9326638822
QCOM [ 0.31339  1.       1.      -0.71784] 76028.03857897197
MA [ 0.58285  0.44653  0.2928  -1.     ] 58441.01333465389


In [ ]:
#!pip install yfinance
#import yfinance as yf


In [ ]:

def hist_data(ticker, t1, t2):
  # p1 - время начала периода
  # p2 - время конца периода
  # форма t = [число, месяц, год]
  p1 = datetime.datetime(t1[2], t1[1], t1[0], 23, 59)
  p2 = datetime.datetime(t2[2], t2[1], t2[0], 23, 59)
  period1 = int(time.mktime(p1.timetuple()))
  period2 = int(time.mktime(p2.timetuple()))
  
  df = yf.download(ticker, p1 , p2)
  b_file = open(f"{ticker}HD.pkl", "wb")
  
  pickle.dump({ticker: np.array(df['Close']), 'Time range': (p2 - p1).days/365}, b_file)
  b_file.close()
  
  return {ticker: df['Close'], 'Time range': (p2 - p1).days/365}
  #return df



  

In [ ]:
tick = 'SPY'
t1 = [30, 4 , 2010]
t2 = [27, 4 , 2022]

In [ ]:
y = hist_data(tick, t1, t2)

[*********************100%***********************]  1 of 1 completed


In [17]:
# Возвращает оценённый параметр vbar, цену начала периода!!!! и длину интервала
def par_teta_est(ticker):
  b_file = open(f"{ticker}HD.pkl", "rb")
  data = pickle.load(b_file)
  b_file.close()
  v = np.array(data[ticker]) 
  S0 = v[0]
  return np.sum(np.array([np.log(v[j + 1]/v[j]) for j in range(len(v) - 1)]) ** 2) / data['Time range'], S0 , data['Time range']

In [ ]:
tickers = ['PANW', 'MA','NVDA', 'QCOM', 'AMD', 'BA']

t1 = [30, 4 , 2010]
t2 = [27, 4 , 2022]


In [18]:
# В случае модели Хестона в начале оценивается параметр thetf, а потом через него ищется параметр mu, и далее эти параметры 
#передаются функции калибровки
def corr_matr_X(tickers, r, N):
  v_P = []
  S_0 = []  
  for ticker in tickers:
    k = par_teta_est(ticker)
    S_0.append(k[1])
    v_P.append(k[0])
    T = k[2]
  mu_P = []
  Par = []
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)
  for ind, ticker in enumerate(tickers):
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    data['Parameters Heston'][2] = v_P[ind]
    Par.append(data['Parameters Heston'])
    d = data['Parameters Heston']
    mu_P.append(r + d[0] * (v_P[ind]-d[2]) / (d[4] * d[1]))
  empcorr = empcorrmatr(tickers)
  NoOfSteps = int(T * 252)
  return corr_calibration(Par, S_0, N, NoOfSteps, T, mu_P, empcorr) , empcorr
  

In [19]:
def empcorrmatr(tickers):
  num_tickers = {}
  num = []
  for ind, ticker in enumerate(tickers):
    num_tickers.update({f'{ind}': ticker})
    num.append(ind)
  empcor = np.zeros([len(num), len(num)])
  for i in range(len(num) - 1):
    b_file = open(f"{num_tickers[f'{i}']}HD.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    path1 = data[num_tickers[f'{i}']]
    for j in range(i + 1, len(num)):
      b_file = open(f"{num_tickers[f'{j}']}HD.pkl", "rb")
      data = pickle.load(b_file)
      path2 = data[num_tickers[f'{j}']]
      b_file.close()
      empcor[i, j] = EmpCorr(path1, path2)
      empcor[j, i] = empcor[i, j]
  for i in range(len(num)):
    empcor[i, i] = 1
  return empcor
      

In [20]:
#  по введённым тикерам возвращает параметры откалиброванных моделей 
# и начальную стоимость в нужном формате, чтобы потом их отправлять на вход функции autocallbondprice
def par_data_final(tickers):
  Par = []
  S0 = []
  for ticker in tickers:
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    Par.append(data['Parameters Heston'])
    b_file = open(f"{ticker}HD.pkl", "rb")
    data = pickle.load(b_file)
    S0.append(np.array(data[ticker])[-1:])
    b_file.close()
  return Par, S0

In [ ]:
# калибровка модели Хестона
def Calibration_Heston(CP, S0, r, N, L, bounds, optimizer, market, expiration, strikes):
    # CP   - тип опциона
    # S0   - начальная цена базового актива
    # r    - процентная ставка (постоянная)
    # N    - число элементов в разложении
    # L    - определяет величину отсечения пространства (L=8 или L=10)
    # bounds - границы для параметров
    # optimizer - тип оптимизации : в данной работе применяется дифференциальная эволюция
    # market - данные, по которым калибруют (даны в виде подразумеваемых волатильностей)
    # expiration - вектор времён до экспирации
    # strikes - вектор страйков
    
    
    # функция ошибок 
    def Err_fun(x):
        # внутри суммирование идёт по страйкам при фиксированной экспирации, а далее идёт суммирвоание по экспирациям
        return np.sum(np.array([np.sum((CallPutOptionPriceCOSMthd(x, CP, S0, r, expiration[i], strikes, N, L).reshape([1,len(strikes)]) - np.array(market[i]).reshape([1,len(strikes)])) ** 2) for i in range(len(expiration))]))
    # оптимизация при заданных границах
    res = optimizer(Err_fun, bounds)
    return res.x, res.fun
    

In [ ]:
tickers = ['BA', 'NVDA']
CP  = OptionType.CALL
r   = 0.0028
N   = 500
L   = 12
bounds = [(0.5, 0.5), (0.01, 1.0), (0.01, 1.0), (0.01, 1.0), (-1.0, 1.0)]
#x0 = [0.3, 0.3, 0.2, -0.7]
for i in tickers:
  a_file = open(f"{i}.pkl", "rb")
  data = pickle.load(a_file)
  a_file.close()
  #optimize.minimize
  #optimize.differential_evolution
  par = Calibration_Heston(CP, data['S0'], r, N, L, bounds, optimize.differential_evolution, data['Option data'], data['Expirations'], data['Strikes'])
  print(i, par[0], par[1])
  a_file = open(f"{i}.pkl", "wb")
  data.update({'Parameters Heston': par[0]})
  pickle.dump(data, a_file)
  a_file.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3

BA [ 0.5      0.26024  0.22354  0.20461 -0.99992] 34.219703694606174
NVDA [ 0.5      0.48803  0.50078  0.3773  -0.99501] 1271.8409984451669


In [21]:
N1 = 50
r = 0.0028

N2 = 10000


coupon = 4.5
barier = 0.65
autocall = 1
nom = 1000
ins = 0.01



#assets = [['CVX'], ['UNH'], ['BA'], ['NVDA'], ['QCOM'], ['AMD'], ['NVDA', 'QCOM'], ['BA', 'CVX'], ['NVDA', 'AMD', 'QCOM'], ['UNH', 'CVX', 'BA']]
assets = [['NVDA', 'AMD'], ['QCOM', 'AMD'] , ['BA', 'UNH'] , ['CVX', 'UNH']]
Expirations = [1, 2 , 3, 5, 10]


for j in assets:
    res = []
    tickers = j
    b_file = open("results_Heston.pkl", "rb")
    res_voc = pickle.load(b_file)
    b_file.close()
    C = corr_matr_X(tickers, r, N1)
    print(C)
    P = par_data_final(tickers)
    par_model = P[0]
    rho_emp_matr = C[0]
    par_S_0 = P[1]
    for i in Expirations:
        T = i
        NoOfSteps =  T * 252
        dates = [k/4 for k in range(1, T * 4 + 1)]
      
        results = autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N2, dates, coupon, barier, autocall, nom, ins)
        res.append([i, results[0], results[1], results[2], results[3]])
        print(j, [i, results[0], results[1], results[2], results[3]])
    res_voc.update({"".join(j): res})
    b_file = open("results_Heston.pkl", "wb")
    pickle.dump(res_voc, b_file)
    b_file.close()

(array([[1.    , 0.5625],
       [0.5625, 1.    ]]), array([[1.        , 0.53106904],
       [0.53106904, 1.        ]]))
['NVDA', 'AMD'] [1, 962.2028724640522, 0.313, 956.9109032444874, 967.4948416836171]
['NVDA', 'AMD'] [2, 981.6124323057147, 0.267, 975.2396131520434, 987.9852514593861]
['NVDA', 'AMD'] [3, 1003.501005719431, 0.246, 996.6363774400484, 1010.3656339988136]
['NVDA', 'AMD'] [5, 1056.299165498373, 0.206, 1048.704643691548, 1063.893687305198]
['NVDA', 'AMD'] [10, 1126.5621057638225, 0.178, 1116.9701095612186, 1136.1541019664264]
(array([[1.    , 0.5625],
       [0.5625, 1.    ]]), array([[1.        , 0.39870289],
       [0.39870289, 1.        ]]))
['QCOM', 'AMD'] [1, 1003.7456642574202, 0.262, 999.2683131154893, 1008.2230153993512]
['QCOM', 'AMD'] [2, 1015.2251930954468, 0.246, 1009.6111878468001, 1020.8391983440935]
['QCOM', 'AMD'] [3, 1040.6541328674634, 0.217, 1034.5313904058473, 1046.7768753290795]
['QCOM', 'AMD'] [5, 1075.8215811657274, 0.192, 1068.7615658865834, 1082.8

In [23]:
b_file = open("results_Heston.pkl", "wb")
pickle.dump(res_voc, b_file)
b_file.close()

In [24]:
b_file = open("results_Heston.pkl", "rb")
data = pickle.load(b_file)
b_file.close()
data

{'AMD': [[1,
   1036.3254075748544,
   0.157,
   1032.5918228647054,
   1040.0589922850033],
  [2, 1048.5799937636664, 0.136, 1044.1487939681565, 1053.0111935591763],
  [3, 1067.8584617955644, 0.118, 1063.114494126465, 1072.6024294646638],
  [5, 1099.8831155907892, 0.095, 1094.6017131161657, 1105.1645180654127],
  [10, 1147.3818596501703, 0.074, 1140.469498608736, 1154.2942206916046]],
 'BA': [[1, 1070.3297770531783, 0.122, 1067.8611740177682, 1072.7983800885884],
  [2, 1086.4365143776745, 0.108, 1083.1373484161063, 1089.7356803392427],
  [3, 1095.0138867202218, 0.105, 1091.0877058721849, 1098.9400675682587],
  [5, 1116.814339877115, 0.088, 1112.1108895394318, 1121.517790214798],
  [10, 1161.3615695625956, 0.071, 1154.7947658262306, 1167.9283732989607]],
 'BACVX': [[1,
   1044.043004015582,
   0.226,
   1040.6096126018629,
   1047.4763954293012],
  [2, 1054.3250517486085, 0.23, 1049.4991409963154, 1059.1509625009016],
  [3, 1070.6038620757336, 0.223, 1064.975219730808, 1076.23250442065

In [ ]:
tickers = ['CVX', 'UNH', 'NVDA', 'QCOM', 'AMD', 'BA']
for i in tickers:
  a_file = open(f"{i}.pkl", "rb")
  data = pickle.load(a_file)
  print(data)
  a_file.close()

{'Ticker': 'CVX', 'S0': 156.240005, 'Option data': array([[28.  , 22.  , 18.74, 11.37,  8.9 ,  6.2 ,  4.54,  3.2 ,  2.04,
         1.41,  0.9 ,  0.64,  0.27,  0.16,  0.06],
       [28.76, 22.98, 22.35, 13.47, 10.55,  8.29,  6.2 ,  4.6 ,  3.5 ,
         2.25,  1.75,  1.2 ,  0.63,  0.34,  0.06],
       [30.03, 27.47, 20.05, 14.65, 12.4 , 10.35,  7.82,  6.15,  4.81,
         3.8 ,  2.9 ,  2.2 ,  1.32,  0.76,  0.18],
       [30.  , 26.2 , 22.  , 16.45, 12.93, 10.95,  9.05,  7.25,  5.85,
         4.65,  3.65,  2.85,  1.91,  1.24,  0.3 ],
       [31.4 , 38.9 , 23.8 , 20.  , 14.25, 12.05, 10.05,  8.7 ,  6.85,
         5.9 ,  4.85,  3.9 ,  2.49,  1.7 ,  0.57],
       [44.51, 25.68, 24.9 , 18.75, 20.2 , 14.85, 11.2 , 10.8 ,  9.25,
         6.81,  5.7 ,  4.55,  3.3 ,  2.03,  1.36],
       [38.6 , 40.54, 30.01, 19.02, 16.34, 13.5 , 12.65, 10.2 ,  7.75,
         7.25,  6.  ,  5.15,  3.4 ,  4.4 ,  1.13],
       [31.89, 27.9 , 24.89, 19.17, 17.3 , 15.24, 12.85, 10.6 ,  9.28,
         8.1 ,  7.6 ,  5